In [1]:
# 导入常用模块
import numpy as np
import pandas as pd 
import torch
from torch import nn 
import os
from torch.utils.data import Dataset,DataLoader 

import warnings 
warnings.filterwarnings('ignore')


In [2]:
import os
os.getcwd()

'/home/featurize'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '.shell.pre-oh-my-zsh',
 '.virtual_documents',
 '.zcompdump',
 '.zcompdump-featurize-5.8',
 '.python_history',
 '.server.log',
 '.nvm',
 '.ipython',
 '.vim',
 '.lesshst',
 '.jupyter',
 '.nv',
 '.oh-my-zsh',
 '.local',
 '.zshrc',
 '.bash_logout',
 '.profile',
 '.bashrc',
 '.npm',
 '.conda',
 '.keras',
 '.config',
 'finetune-mistral-7B.ipynb',
 '.zsh_history',
 '.cache',
 '.ssh',
 '.yarn',
 '.wget-hsts',
 '.vscode-server',
 'work',
 'train-dir']

In [10]:
#!pip install -r requirements.txt

In [11]:
!pip install -q torch
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install -q peft #Parameter efficient finetuning - for qLora Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training
!pip install jsonlines

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 2.22.0 requires urllib3<2.0, but you have urllib3 2.1.0 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


In [4]:
!nvidia-smi

Sun Dec  3 01:24:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:10:00.0 Off |                  Off |
| 30%   25C    P8     6W / 300W |      0MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

----------------

### 导入库

In [5]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

2023-12-03 01:24:31.912598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 01:24:32.589479: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
## 样例数据
{
    "instruction": "Create a function to calculate the sum of a sequence of integers.",
    "input":"[1, 2, 3, 4, 5]",
    "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}

{'instruction': 'Create a function to calculate the sum of a sequence of integers.',
 'input': '[1, 2, 3, 4, 5]',
 'output': '# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum'}

In [27]:
# Load your dataset (replace 'your_dataset_name' and 'split_name' with your actual dataset information)
# dataset = load_dataset("your_dataset_name", split="split_name")
dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")

In [29]:
# this function is used to output the right formate for each row in the dataset
def create_text_row(instruction, output, input):
    text_row = f"""<s>[INST] {instruction} here are the inputs {input} [/INST] \\n {output} </s>"""
    #text_row = f"<s>[INST] {instruction} here are the inputs {input} [/INST] \\n {output} </s>"
    return text_row

# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in dataset:
            json_object = {
                "text": create_text_row(item["instruction"], item["input"] ,item["output"]),
                "instruction": item["instruction"],
                "input": item["input"],
                "output": item["output"]
            }
            output_jsonl_file.write(json.dumps(json_object) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset_2new.jsonl")

In [20]:
{
    "text":"<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]\\\
    # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>",
    "instruction":"Create a function to calculate the sum of a sequence of integers",
    "input":"[1, 2, 3, 4, 5]",
    "output":"# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}

{'text': '<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]\\    # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>',
 'instruction': 'Create a function to calculate the sum of a sequence of integers',
 'input': '[1, 2, 3, 4, 5]',
 'output': '# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum'}

In [29]:
import jsonlines

with jsonlines.open('training_dataset_2.jsonl') as reader:
    for obj in reader:
        # 处理每个 JSON 对象
        print(obj)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
## 训练集
#train_dataset = load_dataset('json', data_files='./training_dataset_example.jsonl',# split='train',
#                            features={'text': 'string', 'instruction': 'string', 'input': 'string', 'output': 'string'})
train_dataset = load_dataset('json', data_files='./training_dataset_2.jsonl', split='train')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1063.73it/s]
Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [32]:
new_model = "mistralai-Code-Instruct" #set the name of the new model

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [33]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MitsralAi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json: 100%|██████████| 571/571 [00:00<00:00, 1.71MB/s]
pytorch_model.bin.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 3.73MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   0%|          | 0.00/9.94G [00:00<?, ?B/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   0%|          | 41.9M/9.94G [00:00<00:26, 367MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   1%|          | 94.4M/9.94G [00:00<00:22, 446MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   2%|▏         | 157M/9.94G [00:00<00:19, 513MB/s] 
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   2%|▏         | 210M/9.94G [00:00<00:19, 498MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   3%|▎         | 283M/9.94G [00:00<00:19, 508MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   3%|▎         | 346M/9.94G [00:00<00:17, 537MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   4%|▍         | 409M/9.94G [00:00<00:17, 542MB/s]
(…)hjTZRWN16W9g__&Key-Pair-Id=KVTP0A1DKRTAX:   5%|▍         | 472M/9.94G [00:00<00:17, 536M

In [37]:
eval_prompt = """Print hello world in python c and c++"""

# import random
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

Print hello world in python c and c++

Python:
```python
print("Hello, World!")
```

C:
```c
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

C++:
```c++
#include <iostream>

int main() {
    std::cout << "Hello, World!\n";
    return 0;
}
```


### Fine-Tuning with qLora and Supervised Finetuning

In [40]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=100, # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize the SFTTrainer for fine-tuning
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map: 100%|██████████| 121959/121959 [00:11<00:00, 10193.89 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### Lets start the training process

In [46]:
torch.cuda.empty_cache()

In [41]:
# Start the training process
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 47.54 GiB total capacity; 45.98 GiB already allocated; 42.00 MiB free; 46.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF